In [1]:
'''# Guidelines for ETL Project

This document contains guidelines, requirements, and suggestions for Project 1.

## Team Effort

Due to the short timeline, teamwork will be crucial to the success of this project! Work closely with your team through all phases of the project to ensure that there are no surprises at the end of the week.

Working in a group enables you to tackle more difficult problems than you'd be able to working alone. In other words, working in a group allows you to **work smart** and **dream big**. Take advantage of it!

## Project Proposal

Before you start writing any code, remember that you only have one week to complete this project. View this project as a typical assignment from work. Imagine a bunch of data came in and you and your team are tasked with migrating it to a production data base.

Take advantage of your Instructor and TA support during office hours and class project work time. They are a valuable resource and can help you stay on track.

## Finding Data

Your project must use 2 or more sources of data. We recommend the following sites to use as sources of data:

* [data.world](https://data.world/)

* [Kaggle](https://www.kaggle.com/)

You can also use APIs or data scraped from the web. However, get approval from your instructor first. Again, there is only a week to complete this!

## Data Cleanup & Analysis

Once you have identified your datasets, perform ETL on the data. Make sure to plan and document the following:

* The sources of data that you will extract from.

* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).

* The type of final production database to load the data into (relational or non-relational).

* The final tables or collections that will be used in the production database.

You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report

At the end of the week, your team will submit a Final Report that describes the following:

* **E**xtract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).

* **T**ransform: what data cleaning or transformation was required.

* **L**oad: the final database, tables/collections, and why this was chosen.

Please upload the report to Github and submit a link to Bootcampspot.

- - -
'''

"# Guidelines for ETL Project\n\nThis document contains guidelines, requirements, and suggestions for Project 1.\n\n## Team Effort\n\nDue to the short timeline, teamwork will be crucial to the success of this project! Work closely with your team through all phases of the project to ensure that there are no surprises at the end of the week.\n\nWorking in a group enables you to tackle more difficult problems than you'd be able to working alone. In other words, working in a group allows you to **work smart** and **dream big**. Take advantage of it!\n\n## Project Proposal\n\nBefore you start writing any code, remember that you only have one week to complete this project. View this project as a typical assignment from work. Imagine a bunch of data came in and you and your team are tasked with migrating it to a production data base.\n\nTake advantage of your Instructor and TA support during office hours and class project work time. They are a valuable resource and can help you stay on track.

In [37]:
# Dependencies
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import itertools
from pprint import pprint
from config import username, password

In [38]:
###EXTRACT

# Read .csv data into DataFrames
dfPop2000 = pd.DataFrame # Population by zipcode, 2000
csvFile = 'Resources/population_by_zip_2000.csv'
dfPop2000 = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfPop2010 = pd.DataFrame # Population by zipcode, 2010
csvFile = 'Resources/population_by_zip_2010.csv'
dfPop2010 = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfBars = pd.DataFrame # Bars dataset
csvFile = 'Resources/8260_1.csv'
dfBars = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)

In [39]:
###TRANSFORM

# Bars DataFrame - dirty data clean-up! (Need to be able to merge on 'zipcode') 
dfBars.dropna(subset=['postalCode'],inplace=True) # Drop rows with no zipcode listed
dfBars['postalCode'] = dfBars['postalCode'].str.zfill(5) # Prepend '0' if zipcode less than 5 digits long
dfBars['postalCode'] = dfBars['postalCode'].str.slice(start=0,stop=5) # Keep only first 5 digits of zipcode
bars_columns = [u'postalCode',u'name'] # the 'u' before column name -should- make sure all data is utf-8 compliant
bars_clean = dfBars[bars_columns].copy()
dfBars=bars_clean.rename(columns={'postalCode':'zipcode'})
dfBars.head()

zipcode                     name
1   53212      The Waterfront Cafe
2   93612        559 Local Brewing
3   53515                Shack The
4   47130         Flat12 Bierworks
5   98072  Redhook Ale Brewery Inc

In [41]:
# Create new DataFrame to hold count of bars in each zipcode
ZipBars = dfBars.groupby(['zipcode']).count()
ZipBars.sort_values('name',ascending=False,inplace=True)
ZipBars.rename(columns={'name':'bar_count'},inplace=True)
ZipBars.head()

bar_count
zipcode           
04412           57
93446           51
94558           49
95448           44
43215           42

In [40]:
# Population DataFrames - cleanup
dfPop2000.dropna(subset=['zipcode'],inplace=True) # Drop rows with no zipcode listed
dfPop2010.dropna(subset=['zipcode'],inplace=True) 
dfPop2000['zipcode'] = dfPop2000['zipcode'].str.zfill(5) # Prepend '0' if zip < 5 digits long
dfPop2010['zipcode'] = dfPop2010['zipcode'].str.zfill(5)
dfPop2000['zipcode'] = dfPop2000['zipcode'].str.slice(start=0,stop=5) # Keep only first 5 digits
dfPop2010['zipcode'] = dfPop2010['zipcode'].str.slice(start=0,stop=5)
dfPop2000.head()

minimum_age maximum_age  gender population zipcode          geo_id
0          10          14  female         75   39769  8600000US39769
1          85         NaN  female        130   44047  8600000US44047
2          30          34  female         11   42436  8600000US42436
3          22          24    male         22   99685  8600000US99685
4          67          69    male         69   04239  8600000US04239

In [43]:
# Convert population data to int to enable aggregation
dfPop2000['population'] = dfPop2010['population'].astype(int)
dfPop2010['population'] = dfPop2010['population'].astype(int)

In [45]:
# Create DataFrames to hold total population by zipcode
ZipPop2000 = dfPop2000.groupby(['zipcode'])['population'].sum()
ZipPop2010 = dfPop2010.groupby(['zipcode'])['population'].sum()
ZipPop2000.head()

zipcode
00602    124560
00603    164067
00606     19845
00610     87048
00612    201030
Name: population, dtype: int64

In [46]:
# Create DataFrames to show population grouped by zipcode and gender
GenderPop2000 = dfPop2000.groupby(['zipcode','gender'])['population'].sum()
GenderPop2010 = dfPop2010.groupby(['zipcode','gender'])['population'].sum()
GenderPop2000.head()

zipcode  gender
00601    female     1469
         male       7665
00602    female     5000
         male       7620
00603    female    14015
Name: population, dtype: int64

In [48]:
# Generate dictionaries from DataFrames
bars_dict = {'zipcode','name'}
pop_by_zip2000 = {'zipcode','population'}
pop_by_zip2010 = {'zipcode','population'}
gender2000 = {'zipcode','gender','population'}
gender2010 = {'zipcode','gender','population'}
pop_female2000 = {'zipcode':'population'}
pop_male2000 = {'zipcode':'population'}
pop_female2010 = {'zipcode':'population'}
pop_male2010 = {'zipcode':'population'}

bars_dict = dfBars.to_dict()
pop_by_zip2000 = ZipPop2000.to_dict()
pop_by_zip2010 = ZipPop2010.to_dict()
gender2000 = GenderPop2000.to_dict()
gender2010 = GenderPop2010.to_dict()
for key, value in gender2000.items():
    if key[1] == 'female': pop_female2000.update({key[0]:value})
    if key[1] == 'male': pop_male2000.update({key[0]:value})
for key, value in gender2010.items():
    if key[1] == 'female': pop_female2010.update({key[0]:value})
    if key[1] == 'male': pop_male2010.update({key[0]:value})

In [21]:
### LOAD
# Create connection to Bars_db
rds_connection_string = username + ':' + password + '@127.0.0.1/Bars_db'
engine = create_engine(f'mysql://{rds_connection_string}',echo=False)
conn = engine.connect()
Base = declarative_base()
engine.table_names()


00601 female 1469
00601 male 7665
00602 female 5000
00602 male 7620
00603 female 14015
00603 male 15013
00604 female 11282
00604 male 6977
00606 female 31808
00606 male 8012
00610 female 44880
00610 male 31067
00612 female 9352
00612 male 29973
00616 female 4861
00616 male 7919
00617 female 4360
00617 male 18722
00622 female 6257
00622 male 8823
00623 female 72753
00623 male 6401
00624 female 37988
00624 male 20343
00627 female 4756
00627 male 3361
00631 female 11032
00631 male 5690
00637 female 6402
00637 male 3390
00638 female 32439
00638 male 8849
00641 female 11834
00641 male 13044
00646 female 77436
00646 male 8654
00647 female 35178
00647 male 2141
00650 female 9335
00650 male 17636
00652 female 8091
00652 male 9720
00653 female 19949
00653 male 15426
00656 female 19150
00656 male 32955
00659 female 4499
00659 male 3957
00660 female 3173
00660 male 23695
00662 female 8677
00662 male 24005
00664 female 19145
00664 male 4116
00667 female 36218
00667 male 11141
00669 female 37257
00

01521 male 8110
01522 female 4310
01522 male 3805
01523 female 4348
01523 male 3711
01524 female 3577
01524 male 63713
01527 female 92339
01527 male 5426
01529 female 5998
01529 male 20765
01531 female 5150
01531 male 6516
01532 female 24076
01532 male 3669
01534 female 4649
01534 male 57560
01535 female 9515
01535 male 52616
01536 female 3088
01536 male 7484
01537 female 5322
01537 male 3272
01540 female 36125
01540 male 5503
01541 female 11790
01541 male 68208
01542 female 4492
01542 male 45775
01543 female 4878
01543 male 2740
01545 female 3239
01545 male 6462
01550 female 20476
01550 male 7975
01560 female 12056
01560 male 4189
01562 female 7522
01562 male 22044
01564 female 5346
01564 male 4023
01566 female 4415
01566 male 22521
01568 female 5361
01568 male 12852
01569 female 12140
01569 male 7997
01570 female 4976
01570 male 9328
01571 female 5645
01571 male 2155
01581 female 7774
01581 male 43616
01583 female 61227
01583 male 21560
01585 female 6559
01585 male 7421
01588 female 

02652 male 2917
02653 female 13270
02653 male 13507
02655 female 2718
02655 male 18281
02657 female 16648
02657 male 21690
02659 female 7662
02659 male 4983
02660 female 7775
02660 male 12814
02661 female 10689
02661 male 4856
02664 female 15224
02664 male 4574
02666 female 7349
02666 male 24402
02667 female 11459
02667 male 6476
02668 female 37946
02668 male 7362
02669 female 10549
02669 male 24590
02670 female 9207
02670 male 60922
02671 female 12648
02671 male 7166
02672 female 12647
02672 male 8303
02673 female 4339
02673 male 31384
02675 female 12786
02675 male 12713
02702 female 6062
02702 male 14652
02703 female 10538
02703 male 7796
02713 female 15165
02713 male 4891
02715 female 7861
02715 male 7111
02717 female 4353
02717 male 3669
02718 female 5536
02718 male 5703
02719 female 6097
02719 male 49793
02720 female 6369
02720 male 4768
02721 female 8862
02721 male 7753
02723 female 77124
02723 male 7242
02724 female 19403
02724 male 8487
02725 female 9137
02725 male 7802
02726 f

03582 male 20856
03583 female 3121
03583 male 2899
03584 female 5595
03584 male 2510
03585 female 2253
03585 male 2957
03588 female 3458
03588 male 5489
03590 female 87562
03590 male 3776
03592 female 10816
03592 male 10548
03595 female 14375
03595 male 6755
03598 female 3880
03598 male 5629
035XX female 18288
035XX male 19588
03601 female 9831
03601 male 14790
03602 female 6193
03602 male 8315
03603 female 5568
03603 male 22917
03605 female 46205
03605 male 15119
03607 female 5025
03607 male 10016
03608 female 17258
03608 male 8613
03609 female 7656
03609 male 3670
03741 female 6566
03741 male 6711
03743 female 16500
03743 male 6661
03745 female 27140
03745 male 7496
03748 female 27307
03748 male 5486
03750 female 8043
03750 male 5760
03751 female 5380
03751 male 5006
03752 female 12125
03752 male 8419
03753 female 10776
03753 male 22161
03755 female 2567
03755 male 44438
03765 female 5240
03765 male 14197
03766 female 23140
03766 male 26641
03768 female 35396
03768 male 4836
03770 fe

04478 male 5907
04479 female 10552
04479 male 4458
04487 female 8695
04487 male 13064
04488 female 52240
04488 male 9724
04490 female 6100
04490 male 25824
04493 female 8091
04493 male 6724
04495 female 57143
04495 male 3673
04496 female 33156
04496 male 9568
04497 female 14493
04497 male 31710
044XX female 3494
044XX male 10819
04530 female 1271
04530 male 36994
04535 female 4487
04535 male 37882
04537 female 15717
04537 male 9995
04538 female 3620
04538 male 5552
04539 female 4034
04539 male 5688
04541 female 7371
04541 male 5649
04543 female 32423
04543 male 3383
04544 female 34718
04544 male 4142
04547 female 13376
04547 male 17892
04548 female 11108
04548 male 13166
04551 female 23640
04551 male 5448
04553 female 7665
04553 male 4837
04554 female 6507
04554 male 34338
04555 female 8505
04555 male 17994
04556 female 41623
04556 male 29818
04558 female 7392
04558 male 2815
04562 female 5800
04562 male 12415
04563 female 5167
04563 male 6842
04564 female 10497
04564 male 4382
04568 f

...

97068 male 15128
97070 female 6955
97070 male 4482
97071 female 8533
97071 male 7701
97080 female 20125
97080 male 6007
970XX female 5919
970XX male 35530
97101 female 6080
97101 male 6336
97102 female 5926
97102 male 5163
97103 female 3872
97103 male 3645
97106 female 8999
97106 male 26693
97107 female 5542
97107 male 46101
97108 female 13070
97108 male 6688
97109 female 4906
97109 male 58786
97110 female 6191
97110 male 38495
97111 female 7271
97111 male 20071
97112 female 30740
97112 male 4186
97113 female 17388
97113 male 4259
97114 female 12752
97114 male 6266
97115 female 24950
97115 male 6920
97116 female 4408
97116 male 3430
97117 female 6670
97117 male 2466
97118 female 19037
97118 male 5034
97119 female 28288
97119 male 15487
97121 female 2807
97121 male 6142
97122 female 6120
97122 male 7618
97123 female 5616
97123 male 62863
97124 female 18158
97124 male 2061
97125 female 12859
97125 male 21451
97127 female 5800
97127 male 30845
97128 female 21663
97128 male 25807
97130 fem

97824 male 37495
97825 female 2816
97825 male 8581
97826 female 5362
97826 male 4097
97827 female 6962
97827 male 12515
97828 female 18649
97828 male 2138
97830 female 7393
97830 male 2881
97833 female 27830
97833 male 15834
97834 female 5184
97834 male 11336
97835 female 6616
97835 male 2875
97836 female 4927
97836 male 4176
97837 female 4600
97837 male 41939
97838 female 37256
97838 male 2826
97839 female 4175
97839 male 5173
97840 female 7394
97840 male 4477
97841 female 7935
97841 male 3578
97842 female 6560
97842 male 8531
97843 female 10037
97843 male 5786
97844 female 10976
97844 male 46303
97845 female 8503
97845 male 5010
97846 female 23231
97846 male 4209
97848 female 18076
97848 male 4369
97850 female 12847
97850 male 3395
97856 female 4308
97856 male 26033
97857 female 51117
97857 male 17924
97859 female 9007
97859 male 8533
97862 female 20378
97862 male 8443
97864 female 3893
97864 male 4150
97865 female 5394
97865 male 3453
97867 female 1559
97867 male 7249
97868 female 1

98526 female 8539
98526 male 8487
98527 female 43042
98527 male 5099
98528 female 5309
98528 male 17922
98530 female 6567
98530 male 17727
98531 female 33822
98531 male 58122
98532 female 2838
98532 male 7047
98533 female 7515
98533 male 56167
98535 female 11299
98535 male 62480
98536 female 58373
98536 male 5122
98537 female 25564
98537 male 30139
98538 female 2699
98538 male 9332
98539 female 24808
98539 male 9014
98541 female 23155
98541 male 4434
98542 female 13704
98542 male 42414
98546 female 6013
98546 male 4422
98547 female 2927
98547 male 17244
98548 female 7520
98548 male 20790
98550 female 32126
98550 male 5584
98552 female 4925
98552 male 3546
98555 female 3847
98555 male 7985
98557 female 3887
98557 male 10683
98559 female 35412
98559 male 12525
98560 female 4815
98560 male 9143
98562 female 7877
98562 male 8567
98563 female 8605
98563 male 20571
98564 female 19904
98564 male 36295
98565 female 26874
98565 male 4612
98568 female 35388
98568 male 6868
98569 female 21001
985

99131 female 4524
99131 male 5967
99133 female 7346
99133 male 11650
99134 female 55321
99134 male 16500
99135 female 5574
99135 male 3741
99136 female 14159
99136 male 6663
99137 female 7278
99137 male 124745
99138 female 31361
99138 male 48655
99139 female 13633
99139 male 5816
99140 female 29241
99140 male 6791
99141 female 5149
99141 male 5467
99143 female 4362
99143 male 30026
99146 female 5899
99146 male 5976
99148 female 2147
99148 male 13664
99149 female 5559
99149 male 5269
99150 female 20369
99150 male 45304
99151 female 20966
99151 male 69619
99152 female 2445
99152 male 37506
99153 female 7167
99153 male 22356
99154 female 7348
99154 male 12369
99155 female 26370
99155 male 21237
99156 female 4862
99156 male 23221
99157 female 3310
99157 male 4187
99158 female 5248
99158 male 9079
99159 female 14360
99159 male 4637
99160 female 69540
99160 male 6199
99161 female 3154
99161 male 6116
99163 female 4652
99163 male 8092
99166 female 39620
99166 male 57723
99167 female 5446
9916

99850 female 15688
99850 male 2919
998XX female 22372
998XX male 5634
99901 female 3704
99901 male 14719
99903 female 21438
99903 male 1968
99918 female 2773
99918 male 5817
99919 female 50429
99919 male 9445
99921 female 3608
99921 male 7530
99922 female 6790
99922 male 43066
99923 female 80957
99923 male 44384
99925 female 13460
99925 male 19558
99926 female 3912
99926 male 4585
99927 female 33117
99927 male 7611
99929 female 5842
99929 male 16008
99950 female 3337
99950 male 7736
999XX female 35139
999XX male 2775


00602 female 42248
00602 male 40792
00603 female 56184
00603 male 53194
00606 female 6694
00606 male 6536
00610 female 29824
00610 male 28208
00612 female 70550
00612 male 63470
00616 female 11398
00616 male 10636
00617 female 25540
00617 male 23654
00622 female 8140
00622 male 7566
00623 female 44916
00623 male 41206
00624 female 26816
00624 male 25666
00627 female 36244
00627 male 34074
00631 female 1898
00631 male 1806
00637 female 26438
00637 male 24136
00638 female 19222
00638 male 18660
00641 female 31884
00641 male 30498
00646 female 39738
00646 male 36660
00647 female 6288
00647 male 6056
00650 female 15426
00650 male 14570
00652 female 4786
00652 male 4644
00653 female 13842
00653 male 12772
00656 female 22106
00656 male 20856
00659 female 43314
00659 male 40592
00660 female 18446
00660 male 16054
00662 female 44508
00662 male 42056
00664 female 16796
00664 male 16772
00667 female 26510
00667 male 25426
00669 female 30246
00669 male 29242
00670 female 9910
00670 male 9704
0067

01436 female 3284
01436 male 3388
01438 female 562
01438 male 556
01440 female 19858
01440 male 20836
01450 female 10764
01450 male 10528
01451 female 4776
01451 male 4718
01452 female 4386
01452 male 4378
01453 female 42102
01453 male 39664
01460 female 9268
01460 male 8580
01462 female 10162
01462 male 10010
01463 female 11588
01463 male 11406
01464 female 5918
01464 male 8504
01467 female 332
01467 male 300
01468 female 3994
01468 male 4048
01469 female 6876
01469 male 6726
01473 female 7274
01473 male 7280
01474 female 2150
01474 male 2100
01475 female 10232
01475 male 10308
01501 female 16636
01501 male 15514
01503 female 2896
01503 male 2836
01504 female 9348
01504 male 8704
01505 female 4528
01505 male 4182
01506 female 3422
01506 male 3358
01507 female 13206
01507 male 12756
01510 female 13916
01510 male 13296
01515 female 2152
01515 male 2214
01516 female 8538
01516 male 8462
01518 female 2988
01518 male 2960
01519 female 6720
01519 male 6470
01520 female 14498
01520 male 1349

02559 female 3546
02559 male 3206
02561 female 520
02561 male 558
02562 female 3134
02562 male 2820
02563 female 10816
02563 male 9902
02564 female 322
02564 male 298
02568 female 4078
02568 male 3922
02571 female 10928
02571 male 10034
02575 female 2850
02575 male 2600
02576 female 3934
02576 male 3574
02584 female 6
02584 male 14
02601 female 14334
02601 male 13844
02630 female 1938
02630 male 1762
02631 female 10396
02631 male 8962
02632 female 11380
02632 male 10104
02633 female 4032
02633 male 3736
02635 female 3356
02635 male 3236
02637 female 256
02637 male 250
02638 female 3418
02638 male 2996
02639 female 3010
02639 male 2740
02641 female 558
02641 male 506
02642 female 5280
02642 male 4594
02643 female 38
02643 male 14
02644 female 4058
02644 male 3852
02645 female 10326
02645 male 9000
02646 female 1800
02646 male 1552
02647 female 136
02647 male 122
02648 female 7512
02648 male 7090
02649 female 14990
02649 male 13022
02650 female 980
02650 male 716
02651 female 14
02651 ma

03809 male 3696
03810 female 1496
03810 male 1580
03811 female 6940
03811 male 6566
03812 female 806
03812 male 842
03813 female 3234
03813 male 3266
03814 female 2312
03814 male 2346
03816 female 1380
03816 male 1356
03817 female 558
03817 male 590
03818 female 3880
03818 male 3734
03819 female 4316
03819 male 4438
03820 female 30646
03820 male 29432
03823 female 1662
03823 male 1776
03824 female 15764
03824 male 13512
03825 female 8472
03825 male 8552
03826 female 2498
03826 male 2442
03827 female 3226
03827 male 3074
03830 female 1598
03830 male 1622
03832 female 380
03832 male 410
03833 female 21690
03833 male 20220
03835 female 6832
03835 male 6740
03836 female 1478
03836 male 1500
03837 female 1552
03837 male 1486
03838 female 1054
03838 male 1008
03839 female 3840
03839 male 3630
03840 female 3650
03840 male 3482
03841 female 6374
03841 male 6294
03842 female 15378
03842 male 15312
03844 female 2228
03844 male 2248
03845 female 986
03845 male 996
03846 female 772
03846 male 826


04450 female 1356
04450 male 1380
04451 female 256
04451 male 262
04453 female 760
04453 male 798
04454 female 74
04454 male 64
04455 female 880
04455 male 966
04456 female 2810
04456 male 2892
04457 female 6028
04457 male 5868
04459 female 758
04459 male 788
04460 female 1414
04460 male 1424
04461 female 3110
04461 male 3030
04462 female 4920
04462 male 4750
04463 female 2876
04463 male 2818
04464 female 646
04464 male 722
04468 female 9806
04468 male 9236
04469 female 2412
04469 male 2992
04471 female 572
04471 male 646
04472 female 2140
04472 male 2148
04473 female 7570
04473 male 7750
04474 female 3764
04474 male 3702
04475 female 428
04475 male 408
04476 female 1292
04476 male 1226
04478 female 278
04478 male 354
04479 female 1388
04479 male 1298
04481 female 574
04481 male 544
04485 female 216
04485 male 250
04487 female 902
04487 male 1026
04488 female 1202
04488 male 1202
04489 female 194
04489 male 200
04490 female 260
04490 male 286
04491 female 128
04491 male 152
04492 femal

...

95687 female 59126
95687 male 71066
95688 female 34798
95688 male 33960
95689 female 1254
95689 male 1234
95690 female 2360
95690 male 2706
95691 female 35338
95691 male 34136
95692 female 4890
95692 male 4648
95693 female 6774
95693 male 6742
95694 female 9012
95694 male 9300
95695 female 38734
95695 male 37158
95697 female 446
95697 male 404
95698 female 232
95698 male 240
95699 female 146
95699 male 162
95701 female 918
95701 male 1048
95703 female 848
95703 male 946
95709 female 4756
95709 male 4788
95713 female 9434
95713 male 9556
95714 female 346
95714 male 418
95715 female 100
95715 male 130
95717 female 222
95717 male 240
95720 female 160
95720 male 154
95721 female 30
95721 male 44
95722 female 4562
95722 male 4608
95724 female 10
95724 male 44
95726 female 8738
95726 male 8946
95728 female 310
95728 male 462
95735 female 56
95735 male 80
95736 female 222
95736 male 258
95742 female 7692
95742 male 7806
95746 female 21670
95746 male 21250
95747 female 54210
95747 male 49530
9

96859 male 226
96860 female 254
96860 male 1994
96863 female 2
96863 male 102
97001 female 224
97001 male 234
97002 female 5370
97002 male 5876
97004 female 4358
97004 male 4418
97005 female 24998
97005 male 24814
97006 female 63876
97006 male 62196
97007 female 68350
97007 male 65558
97008 female 30020
97008 male 27918
97009 female 7682
97009 male 7842
97011 female 546
97011 male 618
97013 female 23242
97013 male 22134
97014 female 1260
97014 male 1316
97015 female 20092
97015 male 18950
97016 female 6016
97016 male 6150
97017 female 2888
97017 male 2956
97018 female 2000
97018 male 1904
97019 female 3214
97019 male 3108
97020 female 786
97020 male 796
97021 female 1090
97021 male 1088
97022 female 3664
97022 male 3780
97023 female 9516
97023 male 10148
97024 female 10556
97024 male 9928
97026 female 3654
97026 male 3976
97027 female 12468
97027 male 11794
97028 female 182
97028 male 252
97029 female 304
97029 male 266
97030 female 38146
97030 male 35240
97031 female 18508
97031 male 

97638 female 470
97638 male 544
97639 female 522
97639 male 618
97640 female 96
97640 male 102
97641 female 1274
97641 male 1352
97701 female 59992
97701 male 57994
97702 female 40676
97702 male 40490
97707 female 5730
97707 male 5898
97710 female 102
97710 male 138
97711 female 58
97711 male 64
97712 female 38
97712 male 38
97720 female 4456
97720 male 4604
97721 female 248
97721 male 294
97722 female 86
97722 male 96
97730 female 258
97730 male 246
97731 female 236
97731 male 320
97732 female 134
97732 male 144
97733 female 728
97733 male 812
97734 female 2678
97734 male 2894
97735 female 146
97735 male 184
97736 female 72
97736 male 74
97737 female 464
97737 male 528
97738 female 1944
97738 male 1938
97739 female 10836
97739 male 11200
97741 female 11292
97741 male 12532
97750 female 388
97750 male 384
97751 female 116
97751 male 142
97752 female 30
97752 male 58
97753 female 1982
97753 male 2050
97754 female 18848
97754 male 18300
97756 female 34370
97756 male 32738
97758 female 10

98385 female 444
98385 male 474
98387 female 43482
98387 male 42864
98388 female 6346
98388 male 8670
98390 female 10790
98390 male 10132
98391 female 43620
98391 male 44998
98392 female 3062
98392 male 2926
98394 female 1110
98394 male 1104
98396 female 644
98396 male 680
98402 female 4968
98402 male 7744
98403 female 8054
98403 male 7372
98404 female 32384
98404 male 31788
98405 female 23252
98405 male 22450
98406 female 22970
98406 male 20250
98407 female 20740
98407 male 19030
98408 female 19048
98408 male 18612
98409 female 23454
98409 male 22524
98416 female 1192
98416 male 836
98418 female 10026
98418 male 10112
98421 female 212
98421 male 2404
98422 female 20482
98422 male 19820
98424 female 10074
98424 male 10206
98430 female 12
98430 male 14
98433 female 13388
98433 male 15780
98438 female 124
98438 male 608
98439 female 4786
98439 male 4892
98443 female 5410
98443 male 5332
98444 female 34658
98444 male 33254
98445 female 30466
98445 male 28856
98446 female 10284
98446 male 

99116 male 1460
99117 female 480
99117 male 490
99118 female 794
99118 male 1068
99119 female 1110
99119 male 1298
99121 female 122
99121 male 168
99122 female 3788
99122 male 3758
99123 female 1060
99123 male 1056
99124 female 246
99124 male 234
99125 female 524
99125 male 530
99126 female 858
99126 male 896
99128 female 310
99128 male 332
99129 female 792
99129 male 794
99130 female 866
99130 male 790
99131 female 154
99131 male 186
99133 female 1504
99133 male 1482
99134 female 682
99134 male 604
99135 female 238
99135 male 272
99136 female 34
99136 male 24
99137 female 338
99137 male 348
99138 female 1186
99138 male 1232
99139 female 1298
99139 male 1304
99140 female 468
99140 male 482
99141 female 5170
99141 male 5310
99143 female 632
99143 male 668
99144 female 34
99144 male 52
99146 female 52
99146 male 60
99147 female 180
99147 male 200
99148 female 2116
99148 male 2096
99149 female 206
99149 male 210
99150 female 350
99150 male 410
99151 female 162
99151 male 166
99152 female 

[]

In [33]:
# Drop existing tables
#engine.execute('DROP TABLE IF EXISTS table_name')
#engine.table_names()

In [21]:
metadata = MetaData()
bar_names = Table('bar_names', metadata,
    Column('id', Integer, primary_key=True,nullable=False),
                  Column('zipcode', String(5)),
                  Column('name', String(50))
                 )
pop_by_gender_2000 = Table('pop_by_gender', metadata,
    Column('id', Integer, primary_key=True,nullable=False),
                     Column('zipcode', String(5)),
                      Column('population_male', Integer),
                      Column('population_female', Integer)
                 )
pop_by_gender_2010 = Table('pop_by_gender', metadata,
    Column('id', Integer, primary_key=True,nullable=False),
                     Column('zipcode', String(5)),
                      Column('population_male', Integer),
                      Column('population_female', Integer)
                 )
metadata.create_all(engine,extend_existing=False)

InvalidRequestError: Table 'pop_by_gender' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [5]:
engine.table_names()

['bar_names', 'bars_per_zipcode', 'pop_by_gender']

In [105]:
#Declaration of the class in order to write into the database. This structure is standard and should align with SQLAlchemy's doc.
class Current(Base):
    __tablename__ = 'bar_names'

    id = Column(Integer, primary_key=True)
    zipcode = Column(String(5))
    name = Column(String(50))

    def __repr__(self):
        return "(id='%s', zipcode='%s', name='%s')" % (self.id, self.zipcode, self.name)

tableToWriteTo = 'bar_names'
listToWrite = dfBars.to_dict(orient='records')
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)

# Open the session
Session = sessionmaker(bind=engine)
session = Session()

# Inser the dataframe into the database in one bulk
conn.execute(table.insert(), listToWrite)

# Commit the changes
session.commit()

# Close the session
session.close()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2019' in position 4: ordinal not in range(256)

In [70]:
ZipBars.to_sql(name='bars_per_zipcode', con=engine, if_exists='replace', index=True, index_label='zipcode',dtype={'zipcode':String(5)})
pd.read_sql_query('select * from bars_per_zipcode', con=engine).head()

zipcode  bar_count
0   04412         57
1   93446         51
2   94558         49
3   95448         44
4   43215         42

In [71]:
id = dfBars.index.tolist()

In [72]:
dfBars.to_sql(name='bar_names', con=engine, if_exists='replace',index=False,dtype={'id':Integer,'zipcode':String(5),'name':String(50)})
engine.table_names()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2019' in position 4: ordinal not in range(256)

In [73]:
dfBars.head()

zipcode                     name
1   53212      The Waterfront Cafe
2   93612        559 Local Brewing
3   53515                Shack The
4   47130         Flat12 Bierworks
5   98072  Redhook Ale Brewery Inc

In [ ]:
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.orm import sessionmaker
import pandas as pd


# Set up of the engine to connect to the database
# the urlquote is used for passing the password which might contain special characters such as "/"
engine = create_engine('mysql://root:%s@localhost/db1' % urlquote('weirdPassword*withsp€cialcharacters'), echo=False)
conn = engine.connect()
Base = declarative_base()

#Declaration of the class in order to write into the database. This structure is standard and should align with SQLAlchemy's doc.
class Current(Base):
    __tablename__ = 'tableName'

    id = Column(Integer, primary_key=True)
    Date = Column(String(500))
    Type = Column(String(500))
    Value = Column(Numeric())

    def __repr__(self):
        return "(id='%s', Date='%s', Type='%s', Value='%s')" % (self.id, self.Date, self.Type, self.Value)

# Set up of the table in db and the file to import
fileToRead = 'file.csv'
tableToWriteTo = 'tableName'

# Panda to create a lovely dataframe
df_to_be_written = pd.read_csv(fileToRead)
# The orient='records' is the key of this, it allows to align with the format mentioned in the doc to insert in bulks.
listToWrite = df_to_be_written.to_dict(orient='records')

metadata = sqlalchemy.schema.MetaData(bind=engine,reflect=True)
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)

# Open the session
Session = sessionmaker(bind=engine)
session = Session()

# Inser the dataframe into the database in one bulk
conn.execute(table.insert(), listToWrite)

# Commit the changes
session.commit()

# Close the session
session.close()